# Final data

In [2]:
from plots import before_after_plot, plot_mean_welch
from get_bads import get_bad_epochs, get_bad_channels
from preprocess import preprocess
import numpy as np

## Splitting per participant

In [9]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)
package = np.empty((5,0,19,77))
epo_class = np.empty((0))
np.random.shuffle(participants)
ind_stop = (7,12,17,22,27,32)
# ind_stop = (2,5,8,11,14,17)

np.save('participants.npy', participants)    

for i in range(len(participants)):
    fs, epochs, channels, y = preprocess('./data2/'+participants[i]+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    package = np.concatenate((package, epochs), axis = 1)
    epo_class = np.concatenate((epo_class, y))
    print("DONE, PARTICIPANT: " + participants[i])
    if i in ind_stop:
        np.save('data_no'+str(i+1)+'.npy', package)
        np.save('class_no'+str(i+1)+'.npy', epo_class)
        package = np.empty((5,0,19,77))
        epo_class = np.empty((0))
        print("SAVED PACKAGE, NUMBER: " + str(i))
np.save('channels.npy', np.array(channels))    
print("FINISHED")

DONE, PARTICIPANT: k8
DONE, PARTICIPANT: k10
DONE, PARTICIPANT: k16
SAVED PACKAGE, NUMBER: 2
DONE, PARTICIPANT: k3
DONE, PARTICIPANT: k12
DONE, PARTICIPANT: k6
SAVED PACKAGE, NUMBER: 5
DONE, PARTICIPANT: k13
DONE, PARTICIPANT: k5
DONE, PARTICIPANT: k18
SAVED PACKAGE, NUMBER: 8
DONE, PARTICIPANT: k11
DONE, PARTICIPANT: k15
DONE, PARTICIPANT: k7
SAVED PACKAGE, NUMBER: 11
DONE, PARTICIPANT: k4
DONE, PARTICIPANT: k14
DONE, PARTICIPANT: k9
SAVED PACKAGE, NUMBER: 14
FINISHED


## Mixing participants

In [101]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)

packages = {1 : np.empty((5,0,19,77)), 2: np.empty((5,0,19,77)), 3: np.empty((5,0,19,77)), 4: np.empty((5,0,19,77)), 5: np.empty((5,0,19,77))}
epo_class = {1 : np.empty((0)), 2: np.empty((0)), 3: np.empty((0)), 4: np.empty((0)), 5: np.empty((0))}
package_0k = np.empty((5,0,19,77))
epo_class_0k = np.empty((0))

for participant in participants:
    print(participant)
    # epochs shape = (freq bands:5, n.o. epochs, n.o. channels: 19, n.o. samples: 77)
    fs, epochs, channels, y = preprocess('./data2/'+participant+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    test_count = int(epochs.shape[1]*0.25)
    package_0k = np.concatenate((package_0k, epochs[:,0:test_count,:,:]), axis = 1)
    epo_class_0k = np.concatenate((epo_class_0k, y[0:test_count]))
    train_val_count = epochs.shape[1]-int(epochs.shape[1]*0.25)
    for i in range(0,5):
        q1, q2 = i*train_val_count//5+test_count, (i+1)*train_val_count//5+test_count
        temp = np.concatenate((packages[i+1], epochs[:,q1:q2,:,:]), axis = 1)
        packages[i+1] = temp
        y_temp = np.concatenate((epo_class[i+1], y[q1:q2]))
        epo_class[i+1] = y_temp
    print("DONE, PARTICIPANT: " + participant)
print("FINISHED")

k3
DONE, PARTICIPANT: k3
k4
DONE, PARTICIPANT: k4
k5
DONE, PARTICIPANT: k5
k6
DONE, PARTICIPANT: k6
k7
DONE, PARTICIPANT: k7
k8
DONE, PARTICIPANT: k8
k9
DONE, PARTICIPANT: k9
k10
DONE, PARTICIPANT: k10
k11
DONE, PARTICIPANT: k11
k12
DONE, PARTICIPANT: k12
k13
DONE, PARTICIPANT: k13
k14
DONE, PARTICIPANT: k14
k15
DONE, PARTICIPANT: k15
k16
DONE, PARTICIPANT: k16
k18
DONE, PARTICIPANT: k18
m1
DONE, PARTICIPANT: m1
m2
DONE, PARTICIPANT: m2
m3
DONE, PARTICIPANT: m3
m4
DONE, PARTICIPANT: m4
m5
DONE, PARTICIPANT: m5
m6
DONE, PARTICIPANT: m6
m7
DONE, PARTICIPANT: m7
m8
DONE, PARTICIPANT: m8
m9
DONE, PARTICIPANT: m9
m10
DONE, PARTICIPANT: m10
m11
DONE, PARTICIPANT: m11
m12
DONE, PARTICIPANT: m12
m13
DONE, PARTICIPANT: m13
m14
DONE, PARTICIPANT: m14
m15
DONE, PARTICIPANT: m15
m16
DONE, PARTICIPANT: m16
m17
DONE, PARTICIPANT: m17
m18
DONE, PARTICIPANT: m18
FINISHED


In [109]:
for i in range(1,6):
    np.save('npy_data_mix/data_no'+str(i)+'.npy', packages[i])
    np.save('npy_data_mix/class_no'+str(i)+'.npy', epo_class[i])

In [110]:
np.save('npy_data_mix/data_no0.npy', package_0k)
np.save('npy_data_mix/class_no0.npy', epo_class_0k)

## All participants separately

In [4]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)

for i, participant in enumerate(participants):
    fs, epochs, channels, y = preprocess('./data2/'+participant+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    np.save(f'npy_data_all_separate/data_part{i+1}.npy', epochs)
    np.save(f'npy_data_all_separate/class_part{i+1}.npy', epochs)
    print("DONE, PARTICIPANT: " + participant)
print("FINISHED")

DONE, PARTICIPANT: k3
DONE, PARTICIPANT: k4
DONE, PARTICIPANT: k5
DONE, PARTICIPANT: k6
DONE, PARTICIPANT: k7
DONE, PARTICIPANT: k8
DONE, PARTICIPANT: k9
DONE, PARTICIPANT: k10
DONE, PARTICIPANT: k11
DONE, PARTICIPANT: k12
DONE, PARTICIPANT: k13
DONE, PARTICIPANT: k14
DONE, PARTICIPANT: k15
DONE, PARTICIPANT: k16
DONE, PARTICIPANT: k18
DONE, PARTICIPANT: m1
DONE, PARTICIPANT: m2
DONE, PARTICIPANT: m3
DONE, PARTICIPANT: m4
DONE, PARTICIPANT: m5
DONE, PARTICIPANT: m6
DONE, PARTICIPANT: m7
DONE, PARTICIPANT: m8
DONE, PARTICIPANT: m9
DONE, PARTICIPANT: m10
DONE, PARTICIPANT: m11
DONE, PARTICIPANT: m12
DONE, PARTICIPANT: m13
DONE, PARTICIPANT: m14
DONE, PARTICIPANT: m15
DONE, PARTICIPANT: m16
DONE, PARTICIPANT: m17
DONE, PARTICIPANT: m18
FINISHED
